<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/Reto_Entrega_1_Equipo41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portada
---

<br>
<p align=center>Instituto Tecnológico y de Estudios Superiores de Monterrey</p>
<p align=center>Escuela de Ingeniería y Ciencias</p>
<p align=center>
<br>
<img src="https://github.com/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/tec.png?raw=true" alt="Logo" width="250"/>
</p>
<br>
<p align=center>Maestría en Inteligencia Artificial Aplicada (MNA)</p>
<p align=center>TC4029. Ciencia y Analítica de Datos</p>
<p align=center> Profesor Titular: María de la Paz Rico </p>
<p align=center> <b>Reto Final - Parte I</b> </p>
<br>
<p align=center>Presentan:</p>
<p align=center>A01150742 | Ovalle Alvarado José</p>
<p align=center>A01793023 | Arroyo Chavelas Jorge Luis</p>
<p align=center>16 de Noviembre de 2022</p>
<br>

---

[GitHub Link](https://github.com/PosgradoMNA/actividades-del-projecto-equipo-41/blob/main/Reto_Entrega_1_Equipo41.ipynb)


# Introducción

En este documento, presentamos un análisis de datos de la calidad del agua para aguas subterraneas. En esta primera parte nos enfocaremos en un análisis exploratorio de los datos, pero este trabajo servirá también como base para un modelo de predicción del semaforo de calidad de la misma, por lo que la metodología irá encaminada no sólo al análisis sino a un proyecto de ciencia de datos y aprendizaje automático completo; desde su análisis exploratorio hasta la presentación de un modelo de clasificación para este semaforo.

Preapremos primero nuestro ambiente de análisis:

In [119]:
############################################################################################
# Librerias y funciones:
############################################################################################

# Para obtener los datos
import requests
import zipfile
import io

# Para manipualr datos
import pandas as pd
import numpy as np

# Para visualizaciones:
from tabulate import tabulate
from matplotlib import pyplot as plt
import seaborn as sns

# Para particiones y validaciones
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import learning_curve
#from sklearn.model_selection import validation_curve

# Para transformaciones:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# Para pipelines:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Otras utilidades
import functools
import warnings

%matplotlib inline

Obtengamos los datos:

In [ ]:
############################################################################################
# Llamar los datos:
############################################################################################

# Definir la url de donde obtener los datos:
datos_url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

# Obtener la respuesta:
datos_response = requests.get(datos_url, stream=True)

# Tomar respuesta como zip en cache:
datos_zip = zipfile.ZipFile(io.BytesIO(datos_response.content))

# Extraer todos los archivos del zip en nuestro espacio local:
datos_zip.extractall("./")

# Definir el lugar donde guardamos los archivos:
datos_dir = './Datos_de_calidad_del_agua_2020'

# Definir archivos a utilizar
datos_file = '/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv'

# Impotar los datos:
df = pd.read_csv(datos_dir + datos_file, encoding = "cp1252")

# Confirmemos que tenemos el dataset:
print(f'\n Dataframe con {df.shape} dimensiones')

Ahora, antes de comenzar a revisar los datos, recurramos a un poco de magía en el servidor de origen para llamar también al diccionario correspondiente a estos datos, ésto nos ayudará a dar mejores conclusiones en nuetro análisis exploratirio – Después de todo, ¿No es acaso descubrir lo creado hacer ciencia? Y esto es ciencia de datos, así que usemos lo disponible.

In [ ]:
############################################################################################
# Llamar al diccionario de datos:
############################################################################################

# Definir la url de donde obtener los datos:
diccionario_url = 'http://201.116.60.46/Diccionario_Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

# Obtener la respuesta:
diccionario_response = requests.get(diccionario_url, stream=True)

# Tomar respuesta como zip en cache:
diccionario_zip = zipfile.ZipFile(io.BytesIO(diccionario_response.content))

# Extraer todos los archivos del zip en nuestro espacio local:
diccionario_zip.extractall("./")

# Definir el lugar donde guardamos los archivos:
diccionario_dir = './Diccionario_Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo'

# Definir archivos a utilizar
diccionario_file = '/Diccionario_datos_Subterraneal.csv'

# Impotar los datos:
diccionario = pd.read_csv(diccionario_dir + diccionario_file, encoding = "UTF-8")

In [ ]:
# Filtremos al diccionario por aquellas varialbes que nos explicará:
diccionario_boooleano = diccionario['CAMPO'].isin(list(df.columns))
diccionario = diccionario[diccionario_boooleano]

# Veamos el diccionario:
display(diccionario.head())
print(diccionario.shape)

,CAMPO,DESCRIPCION,TIPO,VALOR_POSIBLE
0,CLAVE,Clave del sitio de monitoreo,Texto,A - Z
1,SITIO,Nombre del sitio de muestreo,Texto,A - Z
9,ACUIFERO,Acu­fero donde se encuentra el sitio de muestreo,Texto,A - Z
10,SUBTIPO,Subtipo de cuerpo de agua donde se encuentra e...,Texto,A - Z
11,LONGITUD,Coordenada de longitud,Numerico,6 decimales


(54, 4)


Excelente, este diccionario y un poco de técnicas exploratorias nos permitirá explicar a mayor detalle los datos que usaremos en esta entrega.

# EDA - Primeros pasos

Como ya hemos definido, el color de semaforo en las aguas será nuestra variable objetivo, lo que convierte a las demás en nuestros *features*. Esto nos permite hacer una separación en una matriz X y un vector y basado en nuestros datos. Sería además conveniente, que realicemos una separación de estos valores en conjuntos de entrenamiento y de prueba, tanto para crear un modelo más robusto en terminos de generalización, como para crear *pipelines* que nos eviten caer en problemas de *data leak* si este modelo llega a estar en producción.

In [ ]:
############################################################################################
# Datos de entrenamiento y prueba:
############################################################################################

# Conjunto con features
X = df.drop('SEMAFORO', axis=1)

# Variable objetivo
y = df[['SEMAFORO']]

# Partir los datos en conjuntos de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

print(f"""
Tamaño de variables de entrenamiento: {X_train.shape} || tamaño de objetivo entrenamiento: {y_train.shape}
Tamaño de variables de prueba: {X_test.shape} || tamaño de objetivo prueba: {y_test.shape}
""")


Tamaño de variables de entrenamiento: (907, 56) || tamaño de objetivo entrenamiento: (907, 1)
Tamaño de variables de prueba: (161, 56) || tamaño de objetivo prueba: (161, 1)



Comencemos por ver los datos:

In [ ]:
# Vista inicial de los datos:
X_train.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
318,DLGUA1194,VALLE DE LA CUEVITA 1,LERMA SANTIAGO PACIFICO,GUANAJUATO,JERECUARO,VALLE DE LA CUEVITA,POZO,-100.611400,20.301370,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
208,DLDUR690,POZO YERBANIS,CUENCAS CENTRALES DEL NORTE,DURANGO,PEÑON BLANCO,PEÑON BLANCO,POZO,-103.880030,24.678480,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
243,DLDUR766,POZO LA LOMA,CUENCAS CENTRALES DEL NORTE,DURANGO,SAN JUAN DEL RIO,SAN JUAN DEL RIO,POZO,-104.455020,24.786160,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
901,OCPBC4331,POZO VG-2 (CNA-04A-001),PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA,ENSENADA,COLONIA VICENTE GUERRERO,POZO,-115.956988,30.755786,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
874,OCNOR4228,CAMPO CORPUS,NOROESTE,SONORA,HERMOSILLO,COSTA DE HERMOSILLO,POZO,-111.645520,28.866650,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI


Utilicemos el método info también para ver los tipos de datos y si estos contienen o no valores estrictamente nulos:

In [ ]:
# Tipo de dato y valroes en las variables:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 907 entries, 318 to 860
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CLAVE                 907 non-null    object 
 1   SITIO                 907 non-null    object 
 2   ORGANISMO_DE_CUENCA   907 non-null    object 
 3   ESTADO                907 non-null    object 
 4   MUNICIPIO             907 non-null    object 
 5   ACUIFERO              907 non-null    object 
 6   SUBTIPO               907 non-null    object 
 7   LONGITUD              907 non-null    float64
 8   LATITUD               907 non-null    float64
 9   PERIODO               907 non-null    int64  
 10  ALC_mg/L              903 non-null    float64
 11  CALIDAD_ALC           903 non-null    object 
 12  CONDUCT_mS/cm         902 non-null    float64
 13  CALIDAD_CONDUC        902 non-null    object 
 14  SDT_mg/L              0 non-null      float64
 15  SDT_M_mg/L           

Acorde a esto, tenemos 5 variables númericas, una de enteros y 50 como texto. Además encontramos que varias de estas contienen nulos ya que no se cumplen los 907 valores para muchos de estas. Sin embargo, para estar seguros, hagamos una validación cruzada contra nuestro diccionario. Primero veamos nuestro diccionario:

In [ ]:
# Valores acorde al diccionario:
diccionario.head(10)

,CAMPO,DESCRIPCION,TIPO,VALOR_POSIBLE
0,CLAVE,Clave del sitio de monitoreo,Texto,A - Z
1,SITIO,Nombre del sitio de muestreo,Texto,A - Z
9,ACUIFERO,Acu­fero donde se encuentra el sitio de muestreo,Texto,A - Z
10,SUBTIPO,Subtipo de cuerpo de agua donde se encuentra e...,Texto,A - Z
11,LONGITUD,Coordenada de longitud,Numerico,6 decimales
12,LATITUD,Coordenada de latitud,Numerico,6 decimales
13,PERIODO,A¤o o periodo en que se realizo el muestreo,Texto,actual
14,ALC_mg/L,"Valor de Alcalinidad Total, en miligramos por ...",Texto,"1 decimal, ND"
15,AS_TOT_mg/L,"Valor de Arsenico Total, en miligramos por litro",Texto,"3 decimales, ND"
16,CD_TOT_mg/L,"Valor de Cadmio Total, en miligramos por litro",Texto,"3 decimales, ND"


Ahora utilicemos el método describe para ver las variables que nuestro dataset clásifico como numéricas:

In [ ]:
df.describe()

,LONGITUD,LATITUD,PERIODO,ALC_mg/L,CONDUCT_mS/cm,SDT_mg/L
count,1068.000000,1068.000000,1068.0,1064.000000,1062.000000,0.0
mean,-101.891007,23.163618,2020.0,235.633759,1138.953013,NaN
std,6.703263,3.887670,0.0,116.874291,1245.563674,NaN
min,-116.664250,14.561150,2020.0,26.640000,50.400000,NaN
25%,-105.388865,20.212055,2020.0,164.000000,501.750000,NaN
50%,-102.174180,22.617190,2020.0,215.527500,815.000000,NaN
75%,-98.974716,25.510285,2020.0,292.710000,1322.750000,NaN
max,-86.864120,32.677713,2020.0,1650.000000,18577.000000,NaN


Bastantes cosas que rescatar de aquí:
* Vemos nuevamente la presencia de nulos
* SDT_mg_L parece que realmente no tiene valores de utilidad
* PERIODO con una desviación de 0 y su resumen de 5 numeros con el mismo valor parece sugerir que no es más que el año, no una variable per-se.
* Además variables que podíamos suponer iban a ser numéricas por su descripción de diccionario no lo son, como el caso de AS_TOT_mg/L.

Estas observaciones hacen más necesario nuestro análisis exploratorio, al parecer hay variables que no necesitamos y otras que podríamos transformar para generar un análisis más robusto y un pipeline para alimentar modelos de aprendizaje automático.

Para lograr esto, creemos primero una función que nos ayude a analizar cada columna. La idea es sencilla, generemos una función que revise la variable y nos devuevla valores únicos y relación contra el semaforo para variables categóricas, boxplots e histograma para variables numéricas y valores promedio contra la varialbe objetivo. Utilizaremos esta función en conjunto al diccionario para obtener estadísticos y visualizaciones que nos permita generar conclusiones sobre los datos. Estas conclusiones, a su vez, nos ayudarán a proponer un pipeline para el tratamiento de datos (nulos y transformaciones) para que se puedan aplicar modelos de aprendizaje automático en una entrega posterior.

``` markdown
Objetivo: Generar una función que nos permita analizar columnas del datast.

      ┌───────────────┐ ┌──────────────────────┐ ┌───────────┐
      │     input     │ │        función       │ │   Output  │
      │               │ │                      │ │           │
      │  columna(str) ├─►  analizar_variable() ├─►    EDA    │
      │               │ │                      │ │           │
      └───────────────┘ └──────────────────────┘ └───────────┘
```

In [121]:
############################################################################################
# Función para analizar variables:
############################################################################################

print(tabulate(
    pd.DataFrame(X_train['ACUIFERO'].value_counts(dropna=False).sort_index()),
    headers = 'keys',
    tablefmt = 'psql')
)

+----------------------------------------+------------+
|                                        |   ACUIFERO |
|----------------------------------------+------------|
| ABREGO                                 |          2 |
| ACTOPAN - SANTIAGO DE ANAYA            |          3 |
| AGUANAVAL                              |          3 |
| AJACUBA                                |          1 |
| ALTO ATOYAC                            |         17 |
| ALTOS DE JALISCO                       |          4 |
| ALZADA-TEPAMES                         |          5 |
| AMAJAC                                 |          2 |
| AREA METROPOLITANA DE MONTERREY        |          2 |
| ARMERIA-TECOMAN-PERIQUILLOS            |          4 |
| ATLIXCO-IZUCAR DE MATAMOROS            |          4 |
| BAJA BABICORA                          |          1 |
| BENITO JUAREZ                          |          3 |
| BUENAVENTURA                           |          1 |
| BUENOS AIRES                           |      

# EDA y transformaciones (por variable)

# Pipeline para ML

# Conclusiones